In [67]:
import pandas as pd
import numpy as np
import os

In [80]:
path_1="v2_secondrun_results/bulk_session1.csv"
path_2= "v2_secondrun_results/bulk_session2.csv"
df_s1 = pd.read_csv(path_1)
df_s2 = pd.read_csv(path_2)
uniqueIDs = df_s1["ppID"].unique()
df_type_check = input("Do you want the data split into session 1 and session 2 dataframes or would you like them stacked?\nPlease answer with either 'split' or 'stacked'.")
destination = "v2_secondrun_results/Session_1 Session_2_split"


# def keep_second_block(df, column="trial_index", marker="0"):
#     if column not in df.columns:
#         return df
    
#     normalized_col = df[column].astype(str).str.strip().str.lower()

#     # Also get numeric equivalents (to catch mixed types)
#     numeric_col = pd.to_numeric(df[column], errors="coerce")

#     # --- Step 3: create boolean mask that catches both text "0" and numeric 0 ---
#     matches = (normalized_col == str(marker).strip().lower()) | (numeric_col == pd.to_numeric(marker, errors="coerce"))

#     positions = df.index[matches].tolist()
#     print("Multiple marker positions exist", positions)

#     if len(positions) >= 2:
#         start_index = positions[-1]
#         new_df = df.loc[start_index:].reset_index(drop=True)
#         print(f"Second block extracted from starting at row {start_index}")
#         return new_df
#     else:
#         print(f"Only Available marker at row: {positions} ")
#         return df

for id in uniqueIDs:
    participant_data = pd.DataFrame([])

    if id == "ppID":
        continue
    else:
        participant_df_session1 = df_s1[df_s1["ppID"] == id]
        participant_df_session2 = df_s2[df_s2["ppID"] == id]
        # lets do the splitting logic first based off the input

        # if not participant_df_session2.empty and not participant_df_session1.empty:
        #     participant_df_session1 = keep_second_block(participant_df_session1)
        #     participant_df_session2 = keep_second_block(participant_df_session2)


        if df_type_check.lower() == "split":
            if not participant_df_session1.empty:
                participant_df_session1.to_csv(f"{destination}/PP_{id}_session_1.csv")
                
            if not participant_df_session2.empty:
                participant_df_session2.to_csv(f"{destination}/PP_{id}_session_2.csv")
                
        # if the dfs need to be stacked
        else:
            if not participant_df_session1.empty and not participant_df_session2.empty:
                participant_data = pd.concat([participant_df_session1, participant_df_session2])
                participant_data.to_csv( f"ppdata/PP_{id}_stacked.csv")
            else:
                continue
            

        




In [81]:
folder = "v2_secondrun_results/Session_1 Session_2_split"
destination = "v2_secondrun_results/Session_1 Session_2_split" ##lets keep it as the folder because we want to overwrite the files in there
datasets = os.listdir(destination)

for data in datasets:
    df = pd.read_csv(f"{destination}/{data}")

    positions = df.index[df["trial_type"] == "preload"].tolist()
    start_index = positions[-1]
    sliced_df = df.iloc[start_index:].reset_index(drop=True)
    sliced_df.to_csv(f"{destination}/{data}")


In [5]:
import pandas as pd
import os

# Paths
path_1 = "datav2session1.csv"
path_2 = "datav2session2.csv"

# Load data
df_s1 = pd.read_csv(path_1)
df_s2 = pd.read_csv(path_2)

# Ensure save directory exists
save_dir = "ppdata"
os.makedirs(save_dir, exist_ok=True)

# Collect all unique participant IDs across both sessions
uniqueIDs = sorted(set(df_s1["ppID"].unique()) | set(df_s2["ppID"].unique()))

# Track missing participants
missing_in_s1 = []
missing_in_s2 = []

for pid in uniqueIDs:
    # Skip invalid labels if any
    if pid == "ppID" or pd.isna(pid):
        continue

    # Filter for participant across both sessions
    s1_data = df_s1[df_s1["ppID"] == pid]
    s2_data = df_s2[df_s2["ppID"] == pid]

    # --- Handle missing cases ---
    if s1_data.empty and not s2_data.empty:
        missing_in_s1.append(pid)
        # Save session 2 data anyway
        s2_data.to_csv(os.path.join(save_dir, f"Data_{pid}_session2.csv"), index=False)
        continue

    elif s2_data.empty and not s1_data.empty:
        missing_in_s2.append(pid)
        # Save session 1 data anyway
        s1_data.to_csv(os.path.join(save_dir, f"Data_{pid}_session1.csv"), index=False)
        continue

    elif s1_data.empty and s2_data.empty:
        # shouldn't happen, but just skip if so
        continue

    # --- If both sessions exist ---
    s1_data.to_csv(os.path.join(save_dir, f"Data_{pid}_session1.csv"), index=False)
    s2_data.to_csv(os.path.join(save_dir, f"Data_{pid}_session2.csv"), index=False)

# --- Log missing participants ---
if missing_in_s1 or missing_in_s2:
    print("⚠️ Missing participants summary:")
    if missing_in_s1:
        print(f"  Present in Session 2 but missing in Session 1: {missing_in_s1}")
    if missing_in_s2:
        print(f"  Present in Session 1 but missing in Session 2: {missing_in_s2}")
else:
    print("✅ All participants found in both sessions.")


⚠️ Missing participants summary:
  Present in Session 2 but missing in Session 1: ['5e7e1679f8a96f4be83e51e4']
  Present in Session 1 but missing in Session 2: ['613b3fb035407b4479f3b4ea', '65fbd867fe676ad5bdb1ae96']
